In [34]:
from __future__ import annotations

import warnings

import pandas as pd
import statsmodels.api as sm

In [110]:
def add_constant(x: pd.Series | pd.DataFrame) -> pd.DataFrame:
    warnings.filterwarnings('ignore')
    x = sm.add_constant(x)
    warnings.resetwarnings()
    return x

## Задание 1

In [111]:
df = pd.read_csv('data/task1.txt', sep='\t', decimal=',')
df.head()

,X,Y
0,0,77.8
1,7,77.1
2,14,78.9
3,21,79.1
4,28,82.7


In [112]:
X = add_constant(df['X'])
Y = df['Y']

In [113]:
model = sm.OLS(Y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     161.7
Date:                Sat, 13 Nov 2021   Prob (F-statistic):           9.88e-18
Time:                        15:01:13   Log-Likelihood:                -182.66
No. Observations:                  55   AIC:                             369.3
Df Residuals:                      53   BIC:                             373.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         73.2010      1.816     40.310      0.0

In [114]:
sm.stats.durbin_watson(results.resid)

0.058253019621764546

In [150]:
"H0 отвергается - автокорреляция значительна."

'H0 отвергается - автокорреляция значительна.'

In [151]:
resid = pd.Series(results.resid)
rho_1 = resid.corr(resid.shift(-1))
rho_1

0.9703799338684133

In [152]:
Y_new = (Y - rho_1 * Y.shift(1)).dropna()
X_new = (X['X'] - rho_1 * X['X'].shift(1)).dropna()
X_new = add_constant(X_new)
Y_new

1     1.604441
2     4.083707
3     2.537023
4     5.942947
5     3.849579
6     2.691048
7     6.196972
8     1.306566
9     2.268060
10    0.759174
11    2.505858
12    2.705858
13    1.411782
14    2.379200
15    2.376238
16    2.173276
17    2.064390
18    2.352541
19    3.149579
20    1.770314
21    2.949579
22    4.664390
23    2.729554
24    3.135478
25    3.753250
26    3.088794
27    3.003604
28    3.515452
29    2.642110
30    2.342110
31    2.433224
32    2.927300
33    3.236186
34    1.353958
35    3.115452
36    2.930262
37    3.539148
38    5.265806
39    4.442818
40    3.793172
41    5.822792
42    4.711653
43    8.064969
44    5.216031
45    8.678233
46    3.641143
47    4.150029
48    6.573726
49    7.968510
50    2.301800
51    3.063294
52    4.448484
53    7.375142
54    3.787698
Name: Y, dtype: float64

In [153]:
new_model = sm.OLS(Y_new, X_new)
new_results = new_model.fit()
print(new_results.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.235
Model:                            OLS   Adj. R-squared:                  0.220
Method:                 Least Squares   F-statistic:                     15.93
Date:                Sat, 13 Nov 2021   Prob (F-statistic):           0.000207
Time:                        15:10:02   Log-Likelihood:                -100.28
No. Observations:                  54   AIC:                             204.6
Df Residuals:                      52   BIC:                             208.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2902      0.858      0.338      0.7

In [154]:
new_param_a = new_results.params[0] / (1 - rho_1)
new_param_a

9.798645424767082

In [155]:
new_param_b = new_results.params[1]
new_param_b

0.2654806734507575

In [158]:
sm.stats.durbin_watson(new_results.resid)

1.7072598939969583

In [159]:
"H0 принимается - автокорреляция незначительна."

'H0 принимается - автокорреляция незначительна.'

In [177]:
not_het_model = sm.OLS(Y / results.predict(), X.apply(lambda x: x / results.predict()))
not_het_results = not_het_model.fit()
print(not_het_results.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     2.000
Date:                Sat, 13 Nov 2021   Prob (F-statistic):              0.163
Time:                        15:25:47   Log-Likelihood:                 69.557
No. Observations:                  55   AIC:                            -135.1
Df Residuals:                      53   BIC:                            -131.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         75.1706      1.513     49.673      0.0